In [21]:
import pandas as pd

In [2]:
campaigns = pd.read_csv('campaigns.csv', parse_dates=['campaign_date'])
email_interactions = pd.read_csv('email_interactions.csv', parse_dates=['email_interaction_date'])
target_profiles = pd.read_csv('target_profiles.csv')
targeted_moviegoers = pd.read_csv('targeted_moviegoers.csv')
transaction_lines = pd.read_csv('transaction_lines.csv', parse_dates=['transaction_line_date'])

In [3]:
#Changing blanks to False in transaction_line_admission
transaction_lines['transaction_line_is_admission'] = transaction_lines['transaction_line_is_admission'].fillna(False)


#Changing the control group count to 0 if its blank
campaigns['campaign_control_group_count'] = campaigns['campaign_control_group_count'].fillna(0).astype(int)

/tmp/ipython-input-3-3922275017.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  transaction_lines['transaction_line_is_admission'] = transaction_lines['transaction_line_is_admission'].fillna(False)


In [4]:
#Checking for null values
print(campaigns.isnull().sum())
print(email_interactions.isnull().sum())
print(targeted_moviegoers.isnull().sum())
print(transaction_lines.isnull().sum())

campaign_id                     0
campaign_description            0
campaign_date                   0
campaign_target_group_count     0
campaign_control_group_count    0
campaign_has_control_group      0
dtype: int64
campaign_id               0
moviegoer_id              0
email_interaction_type    0
email_interaction_date    0
dtype: int64
campaign_id          0
moviegoer_id         0
target_profile_id    0
dtype: int64
transaction_line_id              0
transaction_id                   0
moviegoer_id                     0
transaction_line_total           0
transaction_line_date            0
transaction_line_type            0
transaction_line_is_admission    0
dtype: int64


In [5]:
# Lowercase and strip string columns
for df in [email_interactions, target_profiles, transaction_lines, targeted_moviegoers]:
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip().str.lower()

In [6]:
interaction = email_interactions.merge(
    targeted_moviegoers,
    how='left',
    on=['campaign_id', 'moviegoer_id']
).merge(
    target_profiles,
    how='left',
    on='target_profile_id'
)


interaction.dropna(inplace=True)
#interaction.isnull().sum()
#interaction.info()
interaction.head()


,campaign_id,moviegoer_id,email_interaction_type,email_interaction_date,target_profile_id,target_profile_group
128,22215,509480,email_open,2019-01-12,2.0,target
129,22215,298388,email_open,2019-01-11,2.0,target
130,22215,453394,email_open,2019-01-11,2.0,target
131,22215,205846,email_open,2019-01-11,2.0,target
132,22215,325475,email_open,2019-01-11,2.0,target


In [7]:
interaction = interaction.merge(
    campaigns,
    how='left',
    on='campaign_id'
)

interaction.head()
interaction.info()

#interaction.to_csv('interaction.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469265 entries, 0 to 469264
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   campaign_id                   469265 non-null  int64         
 1   moviegoer_id                  469265 non-null  int64         
 2   email_interaction_type        469265 non-null  object        
 3   email_interaction_date        469265 non-null  datetime64[ns]
 4   target_profile_id             469265 non-null  float64       
 5   target_profile_group          469265 non-null  object        
 6   campaign_description          469265 non-null  object        
 7   campaign_date                 469265 non-null  datetime64[ns]
 8   campaign_target_group_count   469265 non-null  int64         
 9   campaign_control_group_count  469265 non-null  int64         
 10  campaign_has_control_group    469265 non-null  bool          
dtypes: bool(1), d

In [8]:
interactions_10d = interaction[
    (interaction["email_interaction_date"] >= interaction["campaign_date"]) &
    (interaction["email_interaction_date"] <= interaction["campaign_date"] + pd.Timedelta(days=10))
]
interactions_10d.head()
interactions_10d.info()
#print (interactions_10d)
#interactions_10d.to_csv('interactions_10d.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 422190 entries, 0 to 469264
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   campaign_id                   422190 non-null  int64         
 1   moviegoer_id                  422190 non-null  int64         
 2   email_interaction_type        422190 non-null  object        
 3   email_interaction_date        422190 non-null  datetime64[ns]
 4   target_profile_id             422190 non-null  float64       
 5   target_profile_group          422190 non-null  object        
 6   campaign_description          422190 non-null  object        
 7   campaign_date                 422190 non-null  datetime64[ns]
 8   campaign_target_group_count   422190 non-null  int64         
 9   campaign_control_group_count  422190 non-null  int64         
 10  campaign_has_control_group    422190 non-null  bool          
dtypes: bool(1), dateti

In [9]:
# Split by interaction type
email_opens_10_day = interactions_10d[interactions_10d['email_interaction_type'] == 'email_open']
link_click_10_day = interactions_10d[interactions_10d['email_interaction_type'] == 'link_click']

# Group and rename
campaign_email_opens_10_day = email_opens_10_day.groupby('campaign_id')['moviegoer_id'].nunique().reset_index()
campaign_email_opens_10_day.rename(columns={'moviegoer_id': 'campaign_unique_email_opens'}, inplace=True)

campaign_link_click_10_day = link_click_10_day.groupby('campaign_id')['moviegoer_id'].nunique().reset_index()
campaign_link_click_10_day.rename(columns={'moviegoer_id': 'campaign_unique_link_clicks'}, inplace=True)

#campaign_link_click_10_day.head()

In [10]:
labeled = targeted_moviegoers.merge(
    target_profiles,
    how='left',
    on='target_profile_id',)
group_counts = labeled.groupby(['campaign_id', 'target_profile_group'])['moviegoer_id'].nunique().reset_index()

group_counts_pivot = group_counts.pivot(index='campaign_id', columns='target_profile_group', values='moviegoer_id').reset_index()
group_counts_pivot.columns.name = None
group_counts_pivot.rename(columns={
    'target': 'campaign_target_count',
    'control': 'campaign_control_count'
}, inplace=True)

group_counts_pivot.fillna(0, inplace=True)
group_counts_pivot['campaign_target_count'] = group_counts_pivot['campaign_target_count'].astype(int)
group_counts_pivot['campaign_control_count'] = group_counts_pivot['campaign_control_count'].astype(int)

In [11]:
#Gold Layer
#Merging targeted_moviegoers with target_profile
targeted = targeted_moviegoers.merge(
    target_profiles,
    how='left',
    on='target_profile_id'
)

targeted = targeted.merge(
    campaigns [['campaign_id', 'campaign_date']],
    how='inner',
    on=['campaign_id']
)

targeted.head()

,campaign_id,moviegoer_id,target_profile_id,target_profile_group,campaign_date
0,22215,343057,2,target,2019-01-11
1,22215,427863,2,target,2019-01-11
2,22215,151438,2,target,2019-01-11
3,22215,477775,2,target,2019-01-11
4,22215,254919,2,target,2019-01-11


In [12]:
transactions= targeted.merge(
    transaction_lines,
    how='left',
    on=['moviegoer_id']
)

transactions.head()

,campaign_id,moviegoer_id,target_profile_id,target_profile_group,campaign_date,transaction_line_id,transaction_id,transaction_line_total,transaction_line_date,transaction_line_type,transaction_line_is_admission
0,22215,343057,2,target,2019-01-11,32433398.0,21159341.0,0.54,2019-02-06,concession,False
1,22215,343057,2,target,2019-01-11,32433396.0,21159341.0,9.93,2019-02-06,box_office,True
2,22215,343057,2,target,2019-01-11,32433401.0,21159341.0,7.92,2019-02-06,concession,False
3,22215,343057,2,target,2019-01-11,32433400.0,21159341.0,8.46,2019-02-06,concession,False
4,22215,343057,2,target,2019-01-11,32433399.0,21159341.0,3.09,2019-02-06,concession,False


In [13]:
transactions_10d = transactions[
    (transactions["transaction_line_date"] >= transactions["campaign_date"]) &
    (transactions["transaction_line_date"] <= transactions["campaign_date"] + pd.Timedelta(days=10))
]


# Split by group
target_tx = transactions_10d[transactions_10d["target_profile_group"] == "target"]
control_tx = transactions_10d[transactions_10d["target_profile_group"] == "control"]



#transactions_10d.head()
#transactions_10d.to_csv('transactions_10d.csv', index=False)

In [14]:
# Target spend summary
target_summary = target_tx.groupby("campaign_id").agg(
    campaign_target_box_office_spend=("transaction_line_total", lambda x: x[target_tx["transaction_line_type"] == "box_office"].sum()),
    campaign_target_concession_spend=("transaction_line_total", lambda x: x[target_tx["transaction_line_type"] == "concession"].sum()),
    campaign_target_total_spend=('transaction_line_total', 'sum'),
    campaign_target_total_admissions=("transaction_line_is_admission", "sum"),
    campaign_target_count=('moviegoer_id', 'nunique')
).reset_index()

target_summary.head()

,campaign_id,campaign_target_box_office_spend,campaign_target_concession_spend,campaign_target_total_spend,campaign_target_total_admissions,campaign_target_count
0,22166,10244.43,7000.88,17551.56,1071,458
1,22173,2850.40,1154.36,4304.76,320,135
2,22174,7324.09,2705.83,11006.17,675,320
3,22175,33896.70,23952.55,58950.50,3382,1441
4,22176,113685.66,50915.88,169965.29,11573,4995


In [15]:
# Control spend summary
control_summary = control_tx.groupby("campaign_id").agg(
    campaign_control_total_spend=("transaction_line_total", "sum"),
    campaign_control_total_admissions=("transaction_line_is_admission", "sum"),
    campaign_control_count=('moviegoer_id', 'nunique')
).reset_index()

control_summary.head()


,campaign_id,campaign_control_total_spend,campaign_control_total_admissions,campaign_control_count
0,22173,360.15,28,13
1,22174,1075.51,63,33
2,22175,1972.26,121,47
3,22176,1697.28,117,47
4,22213,1242.86,70,33


In [16]:
# Count moviegoers
counts = targeted.groupby(["campaign_id", "target_profile_group"])["moviegoer_id"].nunique().reset_index()
target_counts = counts[counts["target_profile_group"] == "target"][["campaign_id", "moviegoer_id"]].rename(columns={"moviegoer_id": "target_count"})
control_counts = counts[counts["target_profile_group"] == "control"][["campaign_id", "moviegoer_id"]].rename(columns={"moviegoer_id": "control_count"})
control_counts.head()

,campaign_id,control_count
2,22173,341
4,22174,350
6,22175,372
8,22176,380
15,22213,337


In [17]:
# Merge all summaries
summary = target_summary.merge(control_summary, on="campaign_id", how="left")
summary = summary.merge(target_counts, on="campaign_id", how="left")
summary = summary.merge(control_counts, on="campaign_id", how="left")

# Compute per-user spend and uplift
summary["campaign_target_spend_per_moviegoer"] = summary["campaign_target_total_spend"] / summary["target_count"]
summary["campaign_control_spend_per_moviegoer"] = summary["campaign_control_total_spend"] / summary["control_count"]
summary["campaign_spend_uplift_per_moviegoer"] = summary["campaign_target_spend_per_moviegoer"] - summary["campaign_control_spend_per_moviegoer"]


#print(summary.head())

In [18]:

campaign_summary = campaigns.merge(summary, how='left', on='campaign_id') \
    .merge(campaign_email_opens_10_day, how='left', on='campaign_id') \
    .merge(campaign_link_click_10_day, how='left', on='campaign_id')

#campaign_summary = campaign_summary.fillna(0, inplace=True)
#print(campaign_summary.columns.tolist())

#campaign_summary.to_csv('campaign_summary.csv', index=False)

In [20]:
# Columns that should stay as float
float_columns = [
    'campaign_target_total_spend',
    'campaign_control_total_spend',
    'campaign_target_spend_per_moviegoer',
    'campaign_control_spend_per_moviegoer',
    'campaign_spend_uplift_per_moviegoer',
    'campaign_target_box_office_spend',
    'campaign_target_concession_spend'
]

# Columns that should be integers
int_columns = [
    'campaign_target_count', 'campaign_control_count',
    'campaign_target_total_admissions', 'campaign_control_total_admissions',
    'campaign_unique_email_opens', 'campaign_unique_link_clicks',
    'control_count', 'target_count'
]

# Fill NaNs
for col in int_columns:
    campaign_summary[col] = campaign_summary[col].fillna(0).astype(int)

for col in float_columns:
    campaign_summary[col] = campaign_summary[col].fillna(0.0).astype(float)

campaign_summary = campaign_summary.round({col: 2 for col in float_columns})
#campaign_summary.head()
campaign_summary.to_csv('campaign_summary.csv', index=False)
